In [8]:
import pandas as pd 
import os 
import sys

sys.path.append(os.path.abspath("../preprocessing_data/")) # need this to import metadata_functions
from metadata_functions import *

In [27]:
# load selected metadata
# EDIT ME
sessions = [110,111,112,113,114,115,116,117,118,119] 
file_types = ['actions','committees','subjects','titles','cosponsors','sponsors','relatedBills','summaries']

metadata_dict = load_metadata(sessions, file_types, "../data/processed_metadata/")

#### Subjects

In [14]:
# extract subjects from each metadata

for session in metadata_dict.keys():

    metadata_dict[session]['subjects']["subject_names"] = metadata_dict[session]['subjects']["legislativeSubjects"].apply(extract_subject_names)

In [15]:
## count subjects for each session 

subject_count_dfs = {}

for session in sessions:

    subject_count_df = count_subjects(metadata_dict[session]['subjects'])

    subject_count_dfs[session] = subject_count_df

In [16]:
## merge all the subject counts for comparison

subject_count_dfs_combined = pd.concat([df.assign(session=session) for session, df in subject_count_dfs.items()]) # add a column for session and combine all the dataframes

# pivot the table to show values for each session
subject_count_dfs_combined = subject_count_dfs_combined.pivot_table(index="subject",
                                                                    columns="session",
                                                                    values="bill_count",
                                                                    fill_value=0
                                                        ).reset_index().rename_axis(None, axis=1)

subject_count_dfs_combined[[116,117,118,119]] = subject_count_dfs_combined[[116,117,118,119]].astype(int) # reset to integers for readability
subject_count_dfs_combined['total'] = subject_count_dfs_combined.iloc[:, 1:].sum(axis=1) # create total column
subject_count_dfs_combined = subject_count_dfs_combined.sort_values('total', ascending=False).reset_index(drop=True) # sort by total

subject_count_dfs_combined[:10]

,subject,110,111,112,113,114,115,116,117,118,119,total
0,Housing and community development funding,0.0,101.0,54.0,36.0,54.0,67.0,162,111,91,22,698.0
1,Housing finance and home ownership,0.0,128.0,127.0,75.0,48.0,55.0,77,47,34,7,598.0
2,Low- and moderate-income housing,0.0,36.0,29.0,27.0,35.0,42.0,99,62,71,16,417.0
3,Congressional oversight,1.0,20.0,11.0,12.0,20.0,36.0,68,64,57,17,306.0
4,Public housing,12.0,26.0,20.0,13.0,25.0,26.0,67,41,39,11,280.0
5,Government information and archives,0.0,36.0,26.0,17.0,15.0,28.0,58,44,34,6,264.0
6,Government lending and loan guarantees,0.0,57.0,44.0,24.0,16.0,19.0,34,17,23,6,240.0
7,Housing supply and affordability,0.0,21.0,15.0,15.0,11.0,21.0,54,36,26,7,206.0
8,Department of Housing and Urban Development,4.0,29.0,19.0,14.0,15.0,18.0,48,30,17,6,200.0
9,Government studies and investigations,0.0,21.0,16.0,10.0,13.0,21.0,35,34,25,7,182.0


In [17]:
subject_count_dfs_combined

,subject,110,111,112,113,114,115,116,117,118,119,total
0,Housing and community development funding,0.0,101.0,54.0,36.0,54.0,67.0,162,111,91,22,698.0
1,Housing finance and home ownership,0.0,128.0,127.0,75.0,48.0,55.0,77,47,34,7,598.0
2,Low- and moderate-income housing,0.0,36.0,29.0,27.0,35.0,42.0,99,62,71,16,417.0
3,Congressional oversight,1.0,20.0,11.0,12.0,20.0,36.0,68,64,57,17,306.0
4,Public housing,12.0,26.0,20.0,13.0,25.0,26.0,67,41,39,11,280.0
...,...,...,...,...,...,...,...,...,...,...,...,...
641,Computers,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1.0
642,Manufacturing,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,1.0
643,Marine and coastal resources,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,1.0
644,Marine pollution,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,1.0


### Chamber Origination

In [23]:
concatinated_metadata = pd.concat([metadata_dict[session]['metadata'] for session in metadata_dict.keys()])

In [24]:
origination_crosstab = pd.crosstab(concatinated_metadata['data.congress'], concatinated_metadata['data.originChamberCode']).reset_index().rename_axis(None, axis=1) # create crosstab

origination_crosstab['total'] = origination_crosstab.iloc[:,1:].sum(axis=1) # add total column

origination_crosstab

,data.congress,H,S,total
0,110,28,19,47
1,111,165,42,207
2,112,133,47,180
3,113,83,26,109
4,114,77,32,109
5,115,102,39,141
6,116,167,82,249
7,117,156,73,229
8,118,167,79,246
9,119,96,43,139


### Sponsors

In [28]:
concatinated_sponsors = pd.concat([metadata_dict[session]['sponsors'] for session in metadata_dict.keys()])

concatinated_sponsors['congressional_session'] = [x[0] for x in concatinated_sponsors['bill_id'].str.split("-")]


In [29]:
sponsors_crosstab = pd.crosstab(concatinated_sponsors['congressional_session'], concatinated_sponsors['party']).reset_index().rename_axis(None, axis=1) # create crosstab

sponsors_crosstab['total'] = sponsors_crosstab.iloc[:,1:].sum(axis=1) # add total column

sponsors_crosstab

,congressional_session,D,I,R,total
0,110,37,0,10,47
1,111,165,0,42,207
2,112,106,0,74,180
3,113,68,0,41,109
4,114,67,0,42,109
5,115,85,0,56,141
6,116,215,1,33,249
7,117,192,1,36,229
8,118,188,2,56,246
9,119,94,1,44,139


### Committees

In [24]:
committee_count_dfs = {}

for session in sessions:

    committee_count_df = pd.DataFrame(metadata_dict[session]['committees']['name'].value_counts()).reset_index()
    committee_count_df['session'] = session

    committee_count_dfs[session] = committee_count_df

In [25]:
committe_count_combined = pd.concat(committee_count_dfs).pivot_table(index='name',
                                           columns='session',
                                           values='count',
                                           fill_value=0
                            ).reset_index().rename_axis(None, axis=1)

committe_count_combined[[116,117,118,119]] = committe_count_combined[[116,117,118,119]].astype(int) # reset to integers for readability
committe_count_combined['total'] = committe_count_combined.iloc[:, 1:].sum(axis=1) # create total column
committe_count_combined = committe_count_combined.sort_values('total', ascending=False).reset_index(drop=True) # sort by total

committe_count_combined

,name,116,117,118,119,total
0,Financial Services Committee,155,145,149,87,536
1,"Banking, Housing, and Urban Affairs Committee",84,67,68,43,262
2,Judiciary Committee,20,17,18,14,69
3,Ways and Means Committee,7,12,18,9,46
4,Transportation and Infrastructure Committee,7,6,7,6,26
5,Energy and Commerce Committee,3,10,8,4,25
6,Education and Workforce Committee,5,8,6,3,22
7,Appropriations Committee,7,6,5,1,19
8,Veterans' Affairs Committee,6,3,3,3,15
9,Budget Committee,7,3,1,0,11
